In [128]:

import redis

r = redis.Redis(host='localhost', port=6379, db=0)

In [129]:
from counter import get_last_counter

bank_id = f"bank_{get_last_counter()}"
# bank_id = "bank_52"
bank_id

'bank_53'

In [130]:
from json import loads
from IPython.display import display

index = loads(r.get(f"faiss_index:v1::{bank_id}"))
# Write index to JSON file
import json
with open('faiss_index.json', 'w') as f:
    json.dump(index, f, indent=2)
display(index.keys())
display(len(index['id_by_index']))

dict_keys(['id_by_index', 'chunk_by_index', 'faiss_index'])

4960

In [131]:
import os
from llama_stack_client import LlamaStackClient

# Test run
client = LlamaStackClient(
    base_url="http://localhost:5001",
)

response = client.memory.query(
    bank_id=bank_id,
    query=["What is William Falder's occupation?"],
    params={"max_chunks": 10},
)

display(response)

for chunk, score in zip(response.chunks, response.scores):
    print(chunk)
    print(score)

QueryDocumentsResponse(chunks=[Chunk(content='```\n>>> from transformers import AutoConfig, FlaxAutoModelForCausalLM\n\n>>> \n>>> model = FlaxAutoModelForCausalLM.from\\_pretrained("bert-base-cased")\n\n>>> \n>>> model = FlaxAutoModelForCausalLM.from\\_pretrained("bert-base-cased", output\\_attentions=True)\n>>> model.config.output\\_attentions\nTrue\n\n>>> \n>>> config = AutoConfig.from\\_pretrained("./pt\\_model/bert\\_pt\\_model\\_config.json")\n>>> model = FlaxAutoModelForCausalLM.from\\_pretrained(\n...     "./pt\\_model/bert\\_pytorch\\_model.bin", from\\_pt=True, config=config\n... )\n```\n\nAutoModelForMaskedLM\n\nclass transformers.AutoModelForMaskedLM\n\n< source\n\n( argskwargs )\n\nThis is a generic model class that will be instantiated as one of the model classes of the library (with a masked language modeling head) when created with the frompretrained() class method or the fromconfig() class method.\n\nThis class cannot be instantiated directly using\n\n```\n\\_\\_init\\_

Chunk(content='```\n>>> from transformers import AutoConfig, FlaxAutoModelForCausalLM\n\n>>> \n>>> model = FlaxAutoModelForCausalLM.from\\_pretrained("bert-base-cased")\n\n>>> \n>>> model = FlaxAutoModelForCausalLM.from\\_pretrained("bert-base-cased", output\\_attentions=True)\n>>> model.config.output\\_attentions\nTrue\n\n>>> \n>>> config = AutoConfig.from\\_pretrained("./pt\\_model/bert\\_pt\\_model\\_config.json")\n>>> model = FlaxAutoModelForCausalLM.from\\_pretrained(\n...     "./pt\\_model/bert\\_pytorch\\_model.bin", from\\_pt=True, config=config\n... )\n```\n\nAutoModelForMaskedLM\n\nclass transformers.AutoModelForMaskedLM\n\n< source\n\n( argskwargs )\n\nThis is a generic model class that will be instantiated as one of the model classes of the library (with a masked language modeling head) when created with the frompretrained() class method or the fromconfig() class method.\n\nThis class cannot be instantiated directly using\n\n```\n\\_\\_init\\_\\_()\n```\n\n(throws an error)

In [132]:
import pandas as pd
import datasets

dataset = datasets.load_dataset("philschmid/markdown-documentation-transformers")

df = dataset["train"].to_pandas()
df = df.rename(columns={'markdown': 'context'})
df['context_length'] = df['context'].apply(lambda x: len(x))
df.describe()

,context_length
count,386.000000
mean,30982.411917
std,34134.173003
min,129.000000
25%,8419.000000
50%,20803.000000
75%,39813.750000
max,213379.000000


In [133]:
doc_ids = []
titles = []
for i, row in df.iterrows():
    titles.append(row["title"])
    doc_ids.append(i)

print(len(titles))
print(len(doc_ids))
display(titles[:5])
doc_ids[:40]

386
386


['🤗 Transformers',
 'Quick tour',
 'Installation',
 'Pipelines for inference',
 'Load pretrained instances with an AutoClass']

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39]

In [134]:
from tqdm import tqdm

top_k = 10

results = []

for i in tqdm(range(0, len(titles))):
    response = client.memory.query(
        bank_id=bank_id,
        query=[titles[i]],
        params={"max_chunks": top_k},
    )
    res = [int(chunk.document_id) for chunk in response.chunks]
    results.append(res)

results

  5%|███▋                                                                      | 19/386 [00:00<00:04, 91.32it/s]

100%|█████████████████████████████████████████████████████████████████████████| 386/386 [00:20<00:00, 19.02it/s]


[[46, 23, 1, 62, 54, 45, 47, 52, 47, 24],
 [80, 175, 32, 34, 18, 338, 0, 316, 168, 44],
 [168, 316, 167, 80, 47, 83, 2, 0, 237, 264],
 [3, 84, 77, 59, 1, 77, 48, 77, 77, 77],
 [4, 70, 70, 70, 70, 70, 70, 70, 70, 70],
 [287, 195, 312, 312, 147, 257, 255, 48, 255, 329],
 [15, 15, 1, 71, 336, 326, 210, 336, 354, 333],
 [1, 80, 64, 7, 375, 83, 384, 64, 83, 329],
 [8, 8, 28, 384, 7, 31, 83, 29, 29, 34],
 [9, 9, 9, 9, 46, 75, 75, 24, 240, 70],
 [10, 47, 10, 17, 47, 47, 336, 15, 333, 15],
 [11, 24, 23, 63, 62, 1, 11, 54, 24, 46],
 [12, 12, 12, 194, 194, 194, 194, 224, 193, 193],
 [13, 13, 253, 134, 192, 131, 47, 78, 217, 110],
 [14, 286, 215, 120, 308, 360, 157, 315, 310, 312],
 [15, 17, 251, 46, 15, 70, 17, 46, 325, 47],
 [16, 16, 16, 16, 85, 16, 16, 77, 16, 110],
 [10, 47, 17, 10, 15, 47, 47, 333, 47, 17],
 [384, 83, 83, 7, 36, 18, 360, 338, 28, 326],
 [19, 19, 20, 179, 166, 260, 124, 154, 183, 260],
 [20, 20, 72, 20, 72, 72, 20, 20, 20, 72],
 [21, 228, 350, 173, 256, 198, 118, 232, 120, 10

In [143]:
import numpy as np

top_k = 3

res = np.array(results)


targets = np.array(doc_ids)
display(targets.shape)
targets = np.expand_dims(targets, axis=1)
display(targets.shape)
targets = targets == res
display(targets.shape)
targets = targets.astype(float)
display(targets)

targets = targets[:, :top_k]
display(targets.shape)

(386,)

(386, 1)

(386, 10)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

(386, 3)

In [144]:
temp = np.sum(targets, axis=1)
temp = np.divide(temp, top_k)
precision = np.sum(temp) / len(temp)
precision

np.float64(0.49740932642487046)

In [145]:
temp = np.logical_or.reduce(targets, axis=1)
temp = temp.astype(float)
recall = np.sum(temp) / len(temp)
recall

np.float64(0.7979274611398963)

In [146]:
ranks = np.tile(np.arange(1, top_k + 1), (len(titles), 1)).astype(float)
temp = np.copy(targets)
for i in range(len(temp)):
    first_true = True
    for j in range(len(temp[i])):
        if temp[i,j]:
            if first_true:
                first_true = False
            else:
                temp[i,j] = False
reciprocal_ranks = temp / ranks
reciprocal_ranks = np.sum(reciprocal_ranks) / len(temp)
reciprocal_ranks = np.mean(reciprocal_ranks)
reciprocal_ranks

np.float64(0.7055267702936097)